In [86]:
import math
import gc
import time
import copy
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset

from torchtext.vocab import build_vocab_from_iterator

from tqdm import tqdm

import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Seeding for consistency in reproducibility
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [70]:
# Courtesy & more details about dataset https://www.kaggle.com/basilb2s/language-detection
!wget -q https://raw.githubusercontent.com/maqboolkhan/Project-NLP/master/Classification/Language%20Detection.csv

In [87]:
ds = pd.read_csv('Language Detection.csv')

# Printing all available languages in the dataset
ds.Language.unique().tolist()

['English',
 'Malayalam',
 'Hindi',
 'Tamil',
 'Portugeese',
 'French',
 'Dutch',
 'Spanish',
 'Greek',
 'Russian',
 'Danish',
 'Italian',
 'Turkish',
 'Sweedish',
 'Arabic',
 'German',
 'Kannada']

In [88]:
ds = ds.loc[  (ds.Language == 'German') | (ds.Language == 'Dutch')]
train_set, test_set = train_test_split(ds, test_size=0.3, random_state=2022)

In [89]:
trg_langs = ds.Language.unique().tolist()
trg_langs

['Dutch', 'German']

In [90]:
class LangDataset(Dataset):
    def __init__(self, ds, trg_langs, train_vocab=None):
        self.corpus = ds

        if not train_vocab:
            self.src_vocab, self.trg_vocab = self._build_vocab()
        else:
            self.src_vocab, self.trg_vocab = train_vocab

    def __len__(self):
        return len(self.corpus)

    def __getitem__(self, item):
        text = self.corpus.iloc[item].Text
        lang = self.corpus.iloc[item].Language

        return {
            'src': self.src_vocab.lookup_indices(text.lower().split()),
            'trg': self.trg_vocab.lookup_indices([lang])
        }

    def _build_vocab(self):
        src_tokens = self.corpus.Text.str.cat().lower().split()

        src_vocab = build_vocab_from_iterator([src_tokens], specials=["<unk>", "<pad>"])
        src_vocab.set_default_index(src_vocab['<unk>'])

        trg_vocab = build_vocab_from_iterator([trg_langs])

        return src_vocab, trg_vocab

In [91]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, maxlen = 5000):
        super(PositionalEncoding, self).__init__()

        # A tensor consists of all the possible positions (index) e.g 0, 1, 2, ... max length of input
        # Shape (pos) --> [max len, 1]
        pos = torch.arange(0, maxlen).unsqueeze(1)
        pos_encoding = torch.zeros((maxlen, d_model))

        # In the paper, they had 2i in the positional encoding formula
        # where i is the dimension 
        sin_den = 10000 ** (torch.arange(0, d_model, 2)/d_model) # sin for even item of position's dimension
        cos_den = 10000 ** (torch.arange(1, d_model, 2)/d_model) # cos for odd 

        pos_encoding[:, 0::2] = torch.sin(pos / sin_den) 
        pos_encoding[:, 1::2] = torch.cos(pos / cos_den)

        # Shape (pos_embedding) --> [max len, d_model]
        pos_encoding = pos_encoding.unsqueeze(-2)
        # Shape (pos_embedding) --> [max len, 1, d_model]

        self.dropout = nn.Dropout(dropout)

        # We want pos_encoding be saved and restored in the `state_dict`, but not trained by the optimizer
        # hence registering it!
        # Source & credits: https://discuss.pytorch.org/t/what-is-the-difference-between-register-buffer-and-register-parameter-of-nn-module/32723/2
        self.register_buffer('pos_encoding', pos_encoding)

    def forward(self, token_embedding):
        # shape (token_embedding) --> [sentence len, batch size, d_model]

        # Concatenating embeddings with positional encodings
        # Note: As we made positional encoding with the size max length of sentence in our dataset 
        #       hence here we are picking till the sentence length in a batch
        #       Another thing to notice is in the Transformer's paper they used FIXED positional encoding, 
        #       there are methods where we can also learn them
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])


class InputEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model):
        super(InputEmbedding, self).__init__()

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, tokens):
        # shape (tokens) --> [sentence len, batch size]
        # shape (inp_emb) --> [sentence len, batch size, d_model]
        # Multiplying with square root of d_model as they mentioned in the Transformer's paper
        inp_emb = self.embedding(tokens.long()) * math.sqrt(self.d_model)
        return inp_emb

In [92]:
class TransformerClassifier(nn.Module):
    def __init__(self,
                 src_vocab_size,
                 trg_vocab_size,
                 d_model,
                 dropout,
                 n_head,
                 dim_feedforward,
                 n_layers
                ):
        super().__init__()

        self.src_inp_emb = InputEmbedding(src_vocab_size, d_model)
        self.trg_inp_emb = InputEmbedding(trg_vocab_size, d_model)

        self.positional_encoding = PositionalEncoding(d_model, dropout=dropout)

        # Only using Encoder of Transformer model
        encoder_layers = nn.TransformerEncoderLayer(d_model, n_head, dim_feedforward, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)

        self.d_model = d_model
        self.decoder = nn.Linear(d_model, trg_vocab_size)

    def forward(self, x):
        x_emb = self.positional_encoding(self.src_inp_emb(x))
        # Shape (output) -> (Sequence length, batch size, d_model)
        output = self.transformer_encoder(x_emb)
        # We want our output to be in the shape of (batch size, d_model) so that
        # we can use it with CrossEntropyLoss hence averaging using first (Sequence length) dimension 
        # Shape (mean) -> (batch size, d_model)
        # Shape (decoder) -> (batch size, d_model)
        return self.decoder(output.mean(0))

In [93]:
hyp_params = {
    "batch_size": 64,
    "lr": 0.0005,
    "num_epochs": 10,
    "d_model": 512, # Input embedding dimension
    "n_head": 8, # No. of multi-head attention block (aka paralle self-attention layers)
    "n_layers": 3,
    "feedforward_dim": 128,
    "dropout": 0.1
}

In [110]:
def collate_fn(batch, pad_value, device):
    trgs = []
    srcs = []
    for row in batch:
        srcs.append(torch.tensor(row["src"], dtype=torch.long).to(device))
        trgs.append(torch.tensor(row["trg"]).to(device))

    padded_srcs = pad_sequence(srcs, padding_value=pad_value)
    return {"src": padded_srcs, "trg": torch.tensor([trgs]).to(device)}

train_langds = LangDataset(train_set, trg_langs)
test_langds = LangDataset(test_set, trg_langs, (train_langds.src_vocab, train_langds.trg_vocab))

SRC_PAD_IDX = train_langds.src_vocab["<pad>"]

train_dt = DataLoader(train_langds, batch_size=hyp_params["batch_size"], shuffle=
                   True, collate_fn=lambda batch_size: collate_fn(batch_size, SRC_PAD_IDX, device))

test_dt = DataLoader(test_langds, batch_size=hyp_params["batch_size"], shuffle=
                   True, collate_fn=lambda batch_size: collate_fn(batch_size, SRC_PAD_IDX, device))

hyp_params["src_vocab_size"] = len(train_langds.src_vocab)
hyp_params["trg_vocab_size"] = len(trg_langs)

In [111]:
def train_model(model, train_dataloader, criterion, optimizer):
    model.train()
    epoch_loss = 0
    for batch_idx, batch in enumerate(tqdm(train_dataloader)):
        # Clear the accumulating gradients
        optimizer.zero_grad()

        src = batch["src"]  # shape --> [seq len, batch size]
        trg = batch["trg"]  # shape --> [1, batch size]

        # shape (out) --> [batch size, trg size]
        out = model(src)
        loss = criterion(out, trg.squeeze(0))

        loss.backward()

        optimizer.step()
        epoch_loss += loss.detach().cpu()

    return epoch_loss/len(train_dataloader)


def evaluate_model(model, valid_dataloader, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(valid_dataloader):
            src = batch["src"]  # shape --> [seq len, batch size]
            trg = batch["trg"]  # shape --> [1, batch size]

            # shape (out) --> [batch size, trg size]
            out = model(src)
            loss = criterion(out, trg.squeeze(0))

            epoch_loss += loss.detach().cpu()

    return epoch_loss/len(valid_dataloader)

In [112]:
model = TransformerClassifier(hyp_params["src_vocab_size"],
                                hyp_params["trg_vocab_size"],
                                hyp_params["d_model"],
                                hyp_params["dropout"],
                                hyp_params["n_head"],
                                hyp_params["feedforward_dim"],
                                hyp_params["n_layers"]
                                ).to(device)

criterion = nn.CrossEntropyLoss().to(device)

optimizer = optim.Adam(model.parameters(), lr=hyp_params["lr"])

In [113]:
min_el = math.inf
patience = 1
best_model = {}
best_epoch = 0

epoch_loss = 0
for epoch in range(hyp_params["num_epochs"]):
  start = time.time()
  gc.collect()
  torch.cuda.empty_cache()

  epoch_loss = train_model(model, train_dt, criterion, optimizer)
  eval_loss = evaluate_model(model, test_dt, criterion)
  
  
  print(f"Epoch: {epoch+1}, Train loss: {epoch_loss:.5f}, Eval loss: {eval_loss:.5f}. Time {time.time() - start:.2f} secs")

  if eval_loss < min_el:
      best_epoch = epoch+1
      min_el = eval_loss
      best_model = copy.deepcopy(model)
      # torch.save({
      #     'model_state_dict': model.state_dict(),
      #     'optimizer_state_dict': optimizer.state_dict(),
      #     'eval_loss': min_el
      # }, 'model-transformer.pt')

100%|██████████| 12/12 [00:01<00:00,  7.63it/s]


Epoch: 1, Train loss: 0.84613, Eval loss: 0.23620. Time 2.43 secs


100%|██████████| 12/12 [00:01<00:00,  9.59it/s]


Epoch: 2, Train loss: 0.13109, Eval loss: 0.08092. Time 1.69 secs


100%|██████████| 12/12 [00:01<00:00,  9.34it/s]


...

Epoch: 8, Train loss: 0.02446, Eval loss: 0.05691. Time 1.70 secs


100%|██████████| 12/12 [00:01<00:00, 10.16it/s]


Epoch: 9, Train loss: 0.02317, Eval loss: 0.05605. Time 1.59 secs


100%|██████████| 12/12 [00:01<00:00,  9.71it/s]


Epoch: 10, Train loss: 0.03389, Eval loss: 0.06959. Time 1.85 secs


In [114]:
f"Best epoch was {best_epoch} with {min_el} eval loss"

'Best epoch was 3 with 0.04549018293619156 eval loss'

In [115]:
true_labels =[]
pred_labels =[]

for i in test_langds:
    inp = torch.tensor(i['src']).unsqueeze(1).to(device)
    trg = i['trg'][0]

    with torch.no_grad():
        pred = best_model(inp).view(-1).argmax().item()

    true_labels.append(trg)
    pred_labels.append(pred)

In [117]:
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96       162
           1       0.95      0.97      0.96       143

    accuracy                           0.96       305
   macro avg       0.96      0.96      0.96       305
weighted avg       0.96      0.96      0.96       305

